In [50]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup
import csv
import pandas as pd
from skills import Skills
from pprint import pprint

PATH = "/Users/jeanie/Downloads/chromedriver"
driver = webdriver.Chrome(PATH)
url = "https://www.runeclan.com/clan/Elite_Team_Killerz/xp-tracker/1?skill=2&criteria_set1=double_xp_weekend"
driver.get(url)

bracketA = []
bracketB = []
bracketC = []
bracketD = []
total_lvls = []
usr = []

# scrape users
def scrape():
    content = driver.page_source
    soup = BeautifulSoup(content, "html.parser")
    results = soup.find_all("table", class_="regular")[1]

    for item in results.tbody.find_all("tr")[2:]:
        username = item.find_all("td", class_="clan_td clan_rsn2")
        dxp = item.find_all("td", class_="clan_td clan_xpgain_trk")
        
        if not username:
            td = soup.new_tag('td')
            td.string="None"
            username.append(td)
        if not dxp:
            td = soup.new_tag('td')
            td.string="None"
            dxp.append(td)
            
        usr.append(username[0].get_text())
        usr.append(dxp[0].get_text())

# click to the next page    
def next_page():
    link = driver.find_element_by_link_text("next »")
    link.click()

def make_new_gains_dict(res):
    dxp_gains = []
    index = 0
    while index < len(res):
        new_dxp_gains_dict = {'Name': res[index], 'DXP Gained': res[index+1]}
        dxp_gains.append(new_dxp_gains_dict)
        index += 2
    return dxp_gains
        
#remove commas in numbers for calculations and convert string numbers to int type
def remove_commas(list):
    index = 0
    while index < len(list):
        list[index]['Total lvl'] = int(list[index]['Total lvl'].replace(',', ''))
        index += 1
    return list

# check if results name matches clan roster then is placed to appropriate bracket based on total lvl from clan roster list
# check to see who participated in dxp
def brackets(dxp, totals):  
    for i in range(0, len(dxp)):
        for j in range(0, len(totals)):
            if dxp[i]['Name'] == totals[j]['Name']:
                if totals[j]['Total lvl'] < 2000:
                    bracketA.append(dxp[i])
                elif totals[j]['Total lvl'] >= 2000 and totals[j]['Total lvl'] < 2500:
                    bracketB.append(dxp[i])
                elif totals[j]['Total lvl'] >= 2500 and totals[j]['Total lvl'] < 2700:
                    bracketC.append(dxp[i])
                elif totals[j]['Total lvl'] >= 2700:
                    bracketD.append(dxp[i]) 

# open file of clan roster and append info to list of dicts
def read_csv():                    
    with open('total_lvls.csv', newline='', encoding='utf-8-sig') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            total_lvls.append(dict(row))
 
read_csv()       
scrape()
next_page()
scrape()
res = [ele for ele in usr if ele != 'None'] 
dxp_gains = make_new_gains_dict(res)
totals = remove_commas(total_lvls)
brackets(dxp_gains, totals)

pprint(bracketA)
pprint(bracketB)
pprint(bracketC)
pprint(bracketD)
               
# bracket data as dataframes for better viewing                
bracketA_data = pd.DataFrame.from_dict(bracketA, orient='columns')
bracketB_data = pd.DataFrame.from_dict(bracketB, orient='columns')
bracketC_data = pd.DataFrame.from_dict(bracketC, orient='columns')
bracketD_data = pd.DataFrame.from_dict(bracketD, orient='columns')  
driver.quit()

[{'DXP Gained': '16,378,274', 'Name': 'ETK Lykos'},
 {'DXP Gained': '1,152,719', 'Name': 'Alex_853'},
 {'DXP Gained': '150', 'Name': 'HCFireboy509'},
 {'DXP Gained': '0', 'Name': 'ETK marshall'}]
[{'DXP Gained': '77,278,617', 'Name': 'Sheilen'},
 {'DXP Gained': '73,273,535', 'Name': 'Aczinor9'},
 {'DXP Gained': '18,628,314', 'Name': 'HisKageness'},
 {'DXP Gained': '15,271,301', 'Name': 'Gunmaster422'},
 {'DXP Gained': '10,729,125', 'Name': 'ETK Brisingr'},
 {'DXP Gained': '10,043,596', 'Name': 'Daretoachv'},
 {'DXP Gained': '8,841,751', 'Name': 'ETK Onyx'},
 {'DXP Gained': '8,328,116', 'Name': 'Lilo Kuo'},
 {'DXP Gained': '7,590,370', 'Name': 'Walrus Gang'},
 {'DXP Gained': '7,444,024', 'Name': 'Seshomarou'},
 {'DXP Gained': '5,060,114', 'Name': 'Zorrezias'},
 {'DXP Gained': '2,129,908', 'Name': 'Take Fight'},
 {'DXP Gained': '1,993,086', 'Name': 'Combustive'},
 {'DXP Gained': '253,039', 'Name': 'Kairomitsu'},
 {'DXP Gained': '33,659', 'Name': 'Sammie2532'}]
[{'DXP Gained': '64,886,135

In [51]:
bracketA_data['Rank'] = range(1, len(bracketA_data) + 1)
bracketA_data = bracketA_data[list(('Rank', 'Name', 'DXP Gained'))]
bracketA_data

,Rank,Name,DXP Gained
0,1,ETK Lykos,"16,378,274"
1,2,Alex_853,"1,152,719"
2,3,HCFireboy509,150
3,4,ETK marshall,0


In [52]:
bracketB_data['Rank'] = range(1, len(bracketB_data) + 1)
bracketB_data = bracketB_data[list(('Rank', 'Name', 'DXP Gained'))]
bracketB_data

,Rank,Name,DXP Gained
0,1,Sheilen,"77,278,617"
1,2,Aczinor9,"73,273,535"
2,3,HisKageness,"18,628,314"
3,4,Gunmaster422,"15,271,301"
4,5,ETK Brisingr,"10,729,125"
5,6,Daretoachv,"10,043,596"
6,7,ETK Onyx,"8,841,751"
7,8,Lilo Kuo,"8,328,116"
8,9,Walrus Gang,"7,590,370"
9,10,Seshomarou,"7,444,024"


In [53]:
bracketC_data['Rank'] = range(1, len(bracketC_data) + 1)
bracketC_data = bracketC_data[list(('Rank', 'Name', 'DXP Gained'))]
bracketC_data

,Rank,Name,DXP Gained
0,1,Garh22,"64,886,135"
1,2,CorruptVicar,"50,510,735"
2,3,wahtdaef,"43,595,211"
3,4,LethalSeraph,"24,743,228"
4,5,Alhifar,"23,589,710"
5,6,Austiin,"14,737,059"
6,7,Crideceon,"14,301,722"
7,8,N E R V,"11,596,414"
8,9,Ben7196,"11,024,641"
9,10,Foreverr420,"9,965,325"


In [54]:
bracketD_data['Rank'] = range(1, len(bracketD_data) + 1)
bracketD_data = bracketD_data[list(('Rank', 'Name', 'DXP Gained'))]
bracketD_data

,Rank,Name,DXP Gained
0,1,Trimmed,"153,700,127"
1,2,M ichael,"100,741,121"
2,3,Cerene,"79,494,526"
3,4,Icekro,"54,460,641"
4,5,Deathmall,"48,512,062"
5,6,Hera Selene,"44,469,746"
6,7,FrogHam,"42,081,579"
7,8,Sheivot,"41,341,913"
8,9,Madhura01,"40,863,415"
9,10,IceKrystalx,"40,683,897"
